In [9]:
# Import stanza
import stanza
stanza.install_corenlp(dir=corenlp_dir)

# Setting StanfordCoreNLP
import os
import stanfordnlp
import pandas as pd
from tqdm import tqdm
corenlp_dir = './corenlp'
os.environ["CORENLP_HOME"] = corenlp_dir
#from stanza.server import CoreNLPClient
from stanfordnlp.server import CoreNLPClient

# Setting AllenNLP
from allennlp.predictors.predictor import Predictor
from allennlp_models.coref.predictors.coref import CorefPredictor
from allennlp_models.coref.dataset_readers.conll import ConllCorefReader
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

# Initialise CoreNLPClient
nlp = CoreNLPClient(annotators=['kbp'], memory='4G', endpoint='http://localhost:9001', output_format='json')

# Input Text
text = "The 1Malaysia Development Berhad scandal (1MDB scandal) is an ongoing political scandal occurring in Malaysia. In 2015, Malaysia's then Prime Minister Najib Razak  was accused of channelling over RM 2.67 billion (approximately US $700 million) from 1Malaysia Development Berhad (1MDB), a government run strategic development company (masterminded by Low Taek Jho, commonly referred to as Jho Low), to his personal bank accounts. The event triggered widespread criticism among Malaysians, with many calling for Najib Razak's resignation including Mahathir Mohamad, one of Najib Razak's predecessors as Prime Minister, who later defeated Najib Razak in the 2018 general election and returned to power. Anwar Ibrahim, a political leader in opposition to Najib Razak, openly questioned 1MDB's credentials. He told Parliament that, according to records held by the Companies Commission, the company has no business address and no appointed auditor. According to its publicly filed accounts, 1MDB had nearly RM 42 billion (US $11.73 billion) in debt. Some of this debt resulted from a $3 billion state guaranteed 2013 bond issue led by the investment bank Goldman Sachs, who have been reported as receiving fees of up to $300 million for the deal, although the bank disputes this figure. The Malaysian Conference of Rulers called for prompt investigation of the scandal, saying that it was causing a crisis of confidence in Malaysia. After the 2018 election, the newly elected Prime Minister, Mahathir Mohamad, reopened the investigation into the 1MDB scandal. Malaysian authorities barred Najib Razak from leaving the country, then seized cash and valuable items from premises linked to him. Najib Razak was charged with criminal breach of trust, money laundering and abuse of power, while Jho Low was charged with money laundering. The US Department of Justice pursued its own investigation into 1MDB, alleging that more than US $4.5 billion was diverted from 1MDB by Jho Low and other conspirators including officials from Malaysia, Saudi Arabia and the United Arab Emirates. Najib Razak was subsequently found guilty of seven charges connected to SRC International, a dummy corporation associated with 1MDB, and was sentenced to twelve years imprisonment."

# Resolve the Text (Replace pronounces into references)
allennlp_resolved = predictor.coref_resolved(text)

# Annotate relations between triples
annotated = nlp.annotate(allennlp_resolved)

# Extract the annotated texts
annotated_sentences = annotated['sentences']
rows = []
for s in tqdm(range(len(annotated_sentences))):
    for r in range(s):
        try:
            subject = annotated_sentences[s]['kbp'][r]['subject']
            subjectSpan = annotated_sentences[s]['kbp'][r]['subjectSpan']
            relation = annotated_sentences[s]['kbp'][r]['relation']
            objects = annotated_sentences[s]['kbp'][r]['object']
            objectSpan = annotated_sentences[s]['kbp'][r]['objectSpan']
            row = list([subject, subjectSpan, relation, objects, objectSpan])
            rows.append(row)
        except:
            pass
    
# View output in a dataframe
relation_df = pd.DataFrame(rows, columns = ['Subject', 'SubjectSpan', 'Relation', 'Object', 'ObjectSpan'])
relation_df

Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-703a02b1a2144e2d.props -preload kbp


100%|██████████| 13/13 [00:00<00:00, 13748.35it/s]


,Subject,SubjectSpan,Relation,Object,ObjectSpan
0,Najib Razak,"[36, 38]",per:countries_of_residence,Malaysia,"[31, 32]"
1,Mahathir Mohamad,"[21, 23]",per:countries_of_residence,Malaysia,"[11, 12]"
2,Mahathir Mohamad,"[21, 23]",per:title,Prime Minister,"[36, 38]"
3,Anwar Ibrahim,"[0, 2]",per:countries_of_residence,Malaysia,"[9, 10]"
4,Najib Razak,"[14, 16]",per:title,leader,"[5, 6]"
5,Najib Razak,"[14, 16]",per:countries_of_residence,Malaysia,"[9, 10]"
6,Anwar Ibrahim,"[0, 2]",per:employee_or_member_of,Parliament,"[12, 13]"
7,Najib Razak,"[9, 11]",per:title,leader,"[5, 6]"
8,Companies Commission,"[21, 23]",org:alternate_names,1Malaysia Development Berhad,"[24, 27]"
9,Anwar Ibrahim,"[0, 2]",per:title,leader,"[5, 6]"
